In [1]:
!pip install nltk==3.8.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 12.9 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.9.1
    Uninstalling nltk-3.9.1:
      Successfully uninstalled nltk-3.9.1


In [23]:
!pip install transformers

We have a syntheticly generated dataset with 120 examples

In [26]:
import json
from IPython.display import display

An example of dataset items

In [27]:
data = {
    "dataset": [
        {
            "names": [
                {
                    "name": "Emily",
                    "surname": "Johnson"
                }
            ]
        },
        {
            "names": [
                {
                    "name": "Michael",
                    "surname": "Smith"
                },
                {
                    "name": "Sarah",
                    "surname": "Williams"
                }
            ]
        }
    ]
}

In [28]:
with open("/content/updated-names-dataset.json", 'r') as f:
  data = json.load(f)

In [29]:
len(data['dataset'])

120

Lets remove names to symbol that must be ovewritten by LLM

In [30]:
for entry in data["dataset"]:
  for names in entry["names"]:
    names['name'] = 'n'
    names['surname'] = 's'

In [31]:
entry = data['dataset']
names = [name for entry in entry for name in entry['names']]

In [32]:
names[:20]

[{'name': 'n', 'surname': 's'},
 {'name': 'n', 'surname': 's'},
 {'name': 'n', 'surname': 's'},
 {'name': 'n', 'surname': 's'},
 {'name': 'n', 'surname': 's'},
 {'name': 'n', 'surname': 's'},
 {'name': 'n', 'surname': 's'},
 {'name': 'n', 'surname': 's'},
 {'name': 'n', 'surname': 's'},
 {'name': 'n', 'surname': 's'},
 {'name': 'n', 'surname': 's'},
 {'name': 'n', 'surname': 's'},
 {'name': 'n', 'surname': 's'},
 {'name': 'n', 'surname': 's'},
 {'name': 'n', 'surname': 's'},
 {'name': 'n', 'surname': 's'},
 {'name': 'n', 'surname': 's'},
 {'name': 'n', 'surname': 's'},
 {'name': 'n', 'surname': 's'},
 {'name': 'n', 'surname': 's'}]

Now we need to replace 'names' to 'a' (array) and 'name' to 'n', 'surname' to 's'.

Need to do that for simplifying ngram training.


In [33]:
for entry in data["dataset"]:
  print(entry)

{'names': [{'name': 'n', 'surname': 's'}]}
{'names': [{'name': 'n', 'surname': 's'}, {'name': 'n', 'surname': 's'}]}
{'names': [{'name': 'n', 'surname': 's'}, {'name': 'n', 'surname': 's'}, {'name': 'n', 'surname': 's'}]}
{'names': [{'name': 'n', 'surname': 's'}, {'name': 'n', 'surname': 's'}, {'name': 'n', 'surname': 's'}, {'name': 'n', 'surname': 's'}]}
{'names': [{'name': 'n', 'surname': 's'}]}
{'names': [{'name': 'n', 'surname': 's'}, {'name': 'n', 'surname': 's'}, {'name': 'n', 'surname': 's'}, {'name': 'n', 'surname': 's'}, {'name': 'n', 'surname': 's'}]}
{'names': [{'name': 'n', 'surname': 's'}]}
{'names': [{'name': 'n', 'surname': 's'}, {'name': 'n', 'surname': 's'}]}
{'names': [{'name': 'n', 'surname': 's'}, {'name': 'n', 'surname': 's'}, {'name': 'n', 'surname': 's'}, {'name': 'n', 'surname': 's'}, {'name': 'n', 'surname': 's'}, {'name': 'n', 'surname': 's'}]}
{'names': [{'name': 'n', 'surname': 's'}]}
{'names': [{'name': 'n', 'surname': 's'}, {'name': 'n', 'surname': 's'}, {

In [36]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
The token `044` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `044`


In [37]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [38]:
llama_model_name = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(llama_model_name)
llama_model = AutoModelForCausalLM.from_pretrained(llama_model_name, torch_dtype=torch.float16)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [41]:
data["dataset"][0]

{'names': [{'name': 'n', 'surname': 's'}]}

In [42]:
dataset = []
for entry in data["dataset"]:
  str_entry = json.dumps(entry)
  entry_ids = tokenizer(str_entry, return_tensors="pt").input_ids
  dataset.append(entry_ids)

In [50]:
example = dataset[0][0]
example

tensor([128000,   5018,  11654,    794,  62853,    609,    794,    330,     77,
           498,    330,  52515,    794,    330,     82,   9388,  14316])

In [61]:
print(dataset[0].shape)

torch.Size([1, 17])


In [55]:
for indx in range(len(example)):
  print(f"{example[indx]} : " + tokenizer.decode(example[indx]))

128000 : <|begin_of_text|>
5018 : {"
11654 : names
794 : ":
62853 :  [{"
609 : name
794 : ":
330 :  "
77 : n
498 : ",
330 :  "
52515 : surname
794 : ":
330 :  "
82 : s
9388 : "}
14316 : ]}


In [59]:
len(dataset)

120

In [ ]:
{"names": [{"name": "n", "surname": "s"}]}

# Ngram provides 1 token
# LLM: 5018 - {" - json generation begin
# Ngram: 11654 - names
# Ngram provides 1 token
# Ngram: 794 - ":
# Ngram provides 1 token
# Ngram: 62853 - [{"
# Ngram: 609 - name
# Ngram: 330 -  "
# Ngram: 77 - n
# LLM: 5018 11654 794 62853 609 330
#{"names": [{"name": "Igor
# LLM: 498 - ", {"names": [{"name": "Igor",
# Ngram: input: 330 77 498 output 330
# Ngram: input 77 498 330  output: 52515
# Ngram: input 498 330 52515 output: 794
#...
# Ngram: output 82
# LLM: ...
# LLL: output 9388
# Ngram: 14316
# Ngram: </s>

In [62]:
traindata = []
for entry in dataset:
  arr = entry.squeeze().tolist()
  traindata.append(arr)

In [65]:
len(traindata)

120

In [77]:
max = 0
for indx in range(len(traindata)):
  if len(traindata[indx]) > max:
    max = len(traindata[indx])
print(max)

89


In [67]:
print(traindata[0])

[128000, 5018, 11654, 794, 62853, 609, 794, 330, 77, 498, 330, 52515, 794, 330, 82, 9388, 14316]


In [68]:
flattened_characters = [char for seq in traindata for char in seq]

In [69]:
len(flattened_characters)

5568

In [72]:
flattened = list(map(str, flattened_characters))

In [75]:
from nltk.util import ngrams
from nltk.lm import MLE
from nltk.lm.preprocessing import padded_everygram_pipeline

n = 3
train_data, vocab = padded_everygram_pipeline(n, [flattened])

train_data_list = list(train_data)
print("Training data:", train_data_list)
print("Vocabulary:", list(vocab))

train_data, vocab = padded_everygram_pipeline(n, [flattened])

model = MLE(n)
model.fit(train_data, vocab)

print("N-gram counts:", model.counts)

Training data: [<generator object everygrams at 0x7e045c6f7be0>]
Vocabulary: ['<s>', '<s>', '128000', '5018', '11654', '794', '62853', '609', '794', '330', '77', '498', '330', '52515', '794', '330', '82', '9388', '14316', '128000', '5018', '11654', '794', '62853', '609', '794', '330', '77', '498', '330', '52515', '794', '330', '82', '14682', '5324', '609', '794', '330', '77', '498', '330', '52515', '794', '330', '82', '9388', '14316', '128000', '5018', '11654', '794', '62853', '609', '794', '330', '77', '498', '330', '52515', '794', '330', '82', '14682', '5324', '609', '794', '330', '77', '498', '330', '52515', '794', '330', '82', '14682', '5324', '609', '794', '330', '77', '498', '330', '52515', '794', '330', '82', '9388', '14316', '128000', '5018', '11654', '794', '62853', '609', '794', '330', '77', '498', '330', '52515', '794', '330', '82', '14682', '5324', '609', '794', '330', '77', '498', '330', '52515', '794', '330', '82', '14682', '5324', '609', '794', '330', '77', '498', '330',

In [76]:
import json

all_ngram_probs = {}
for context in model.counts[model.order - 1].keys():
    all_ngram_probs[context] = {}

    for word in model.counts[model.order - 1][context].keys():
        prob = model.score(word, context)
        all_ngram_probs[context][word] = prob

json_serializable = {
    "order": model.order,
    "ngram_probs": {
        " ".join(context): {
            word: prob for word, prob in next_words.items()
        }
        for context, next_words in all_ngram_probs.items()
    }
}

with open("ngram_model.json", "w") as f:
    json.dump(json_serializable, f)

In [78]:
def predict_next_id(context):
    for i in range(len(context), 0, -1):
        sub_context = context[-i:]
        try:
            return model.generate(1, text_seed=sub_context)
        except ValueError:
            continue
    return model.generate(1)

In [81]:
context = ['128000', '5018', '11654', '794', '62853']
next_char = predict_next_char(context)
print(f"Context: {context}, Predicted next id: {next_char}")

Context: ['128000', '5018', '11654', '794', '62853'], Predicted next id: 609


In [ ]:
# for entry in data["dataset"]:
#   entry["a"] = entry.pop("names")
#   for item in entry["a"]:
#     item["n"] = item.pop("name")
#     item["s"] = item.pop("surname")

# dataset = []
# for entry in data["dataset"]:
#   dataset.append(entry)
#   print(entry)

In [11]:
dataset[2]

{'a': [{'n': 'n', 's': 's'}, {'n': 'n', 's': 's'}, {'n': 'n', 's': 's'}]}

In [12]:
from nltk.util import ngrams
from nltk.lm import MLE
from nltk.lm.preprocessing import padded_everygram_pipeline

In [13]:
def json_to_characters(json_obj):
    json_str = json.dumps(json_obj, separators=(',', ':'))
    return list(json_str)

In [14]:
character_sequences = [json_to_characters(item) for item in dataset]

In [15]:
print(character_sequences[0])
len(character_sequences)

['{', '"', 'a', '"', ':', '[', '{', '"', 'n', '"', ':', '"', 'n', '"', ',', '"', 's', '"', ':', '"', 's', '"', '}', ']', '}']


120

In [16]:
flattened_characters = [char for seq in character_sequences for char in seq]

In [17]:
len(flattened_characters)

8292

In [18]:
n = 3
train_data, vocab = padded_everygram_pipeline(n, [flattened_characters])

train_data_list = list(train_data)
print("Training data:", train_data_list)
print("Vocabulary:", list(vocab))

train_data, vocab = padded_everygram_pipeline(n, [flattened_characters])

model = MLE(n)
model.fit(train_data, vocab)

print("N-gram counts:", model.counts)

Training data: [<generator object everygrams at 0x7e04e77b3be0>]
Vocabulary: ['<s>', '<s>', '{', '"', 'a', '"', ':', '[', '{', '"', 'n', '"', ':', '"', 'n', '"', ',', '"', 's', '"', ':', '"', 's', '"', '}', ']', '}', '{', '"', 'a', '"', ':', '[', '{', '"', 'n', '"', ':', '"', 'n', '"', ',', '"', 's', '"', ':', '"', 's', '"', '}', ',', '{', '"', 'n', '"', ':', '"', 'n', '"', ',', '"', 's', '"', ':', '"', 's', '"', '}', ']', '}', '{', '"', 'a', '"', ':', '[', '{', '"', 'n', '"', ':', '"', 'n', '"', ',', '"', 's', '"', ':', '"', 's', '"', '}', ',', '{', '"', 'n', '"', ':', '"', 'n', '"', ',', '"', 's', '"', ':', '"', 's', '"', '}', ',', '{', '"', 'n', '"', ':', '"', 'n', '"', ',', '"', 's', '"', ':', '"', 's', '"', '}', ']', '}', '{', '"', 'a', '"', ':', '[', '{', '"', 'n', '"', ':', '"', 'n', '"', ',', '"', 's', '"', ':', '"', 's', '"', '}', ',', '{', '"', 'n', '"', ':', '"', 'n', '"', ',', '"', 's', '"', ':', '"', 's', '"', '}', ',', '{', '"', 'n', '"', ':', '"', 'n', '"', ',', '"', 's'

In [19]:
def predict_next_char(context):
    for i in range(len(context), 0, -1):
        sub_context = context[-i:]
        try:
            return model.generate(1, text_seed=sub_context)
        except ValueError:
            continue
    return model.generate(1)

In [22]:
import json

all_ngram_probs = {}
for context in model.counts[model.order - 1].keys():
    all_ngram_probs[context] = {}

    for word in model.counts[model.order - 1][context].keys():
        prob = model.score(word, context)
        all_ngram_probs[context][word] = prob

json_serializable = {
    "order": model.order,
    "ngram_probs": {
        " ".join(context): {
            word: prob for word, prob in next_words.items()
        }
        for context, next_words in all_ngram_probs.items()
    }
}

with open("ngram_model.json", "w") as f:
    json.dump(json_serializable, f)

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
The token `bellum` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `bellum`


In [ ]:
llama_model_name = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(llama_model_name)
llama_model = AutoModelForCausalLM.from_pretrained(llama_model_name, torch_dtype=torch.float16)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
def is_json(string):
    json_chars = ['{', '}', '[', ']', ':', ',']
    return any(char in string for char in json_chars)

In [ ]:
def speculative_decoding_with_alternation(prompt, max_length=100):
    generated_text = prompt
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    llama_input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    ngram_active = False
    llama_model.to(device)

    for _ in range(max_length):
        if not ngram_active:
            with torch.no_grad():
                outputs = llama_model.generate(
                    llama_input_ids,
                    max_new_tokens=1,
                    do_sample=False,
                    pad_token_id=tokenizer.eos_token_id
                )
            next_token_id = outputs[0, -1].item()
            next_token = tokenizer.decode(next_token_id)
            generated_text += next_token

            print(f"Generated by Llama: {generated_text}")
            if is_json(next_token):
                ngram_active = True

        else:
            context = list(generated_text[-(n - 1):])
            try:
                next_ngram_token = model.generate(1, text_seed=context)
            except ValueError:
                next_ngram_token = None

            if next_ngram_token:
                generated_text += next_ngram_token

                if next_ngram_token == '.':
                    ngram_active = False

            else:
                ngram_active = False

        llama_input_ids = tokenizer(generated_text, return_tensors="pt").input_ids.cuda()

        if generated_text.endswith('}') or generated_text.endswith(tokenizer.eos_token):
            break

    return generated_text

prompt = """In this text find all names and surname and provide an output as json: {'a': [{'n': '.', 's': '.'}]}
An example of text: "John Smith and Alice Brown went to the park."
Output:"{'a': [{'n': 'John', 's': 'Smith'}, {'n': 'Alice', 's': 'Brown'}]}"
Text: "Simon Green opened his bar."
Output:"""
output = speculative_decoding_with_alternation(prompt, max_length=100)
print("Generated JSON:", output)

Generated by Llama: In this text find all names and surname and provide an output as json: {'a': [{'n': '.', 's': '.'}]}
An example of text: "John Smith and Alice Brown went to the park."
Output:"{'a': [{'n': 'John', 's': 'Smith'}, {'n': 'Alice', 's': 'Brown'}]}"
Text: "Simon Green opened his bar."
Output: "
Generated by Llama: In this text find all names and surname and provide an output as json: {'a': [{'n': '.', 's': '.'}]}
An example of text: "John Smith and Alice Brown went to the park."
Output:"{'a': [{'n': 'John', 's': 'Smith'}, {'n': 'Alice', 's': 'Brown'}]}"
Text: "Simon Green opened his bar."
Output: "{'
Generated by Llama: In this text find all names and surname and provide an output as json: {'a': [{'n': '.', 's': '.'}]}
An example of text: "John Smith and Alice Brown went to the park."
Output:"{'a': [{'n': 'John', 's': 'Smith'}, {'n': 'Alice', 's': 'Brown'}]}"
Text: "Simon Green opened his bar."
Output: "{'"n":".","
Generated by Llama: In this text find all names and surna